In [1]:
import pandas as pd
from src.process import Process

In [2]:
remove = pd.read_csv('./data/Fixed Location.csv')

In [58]:
volumn = pd.read_csv('./data/Volume.csv', dtype=str)

In [4]:
p = Process()
p.clean_data()
wd = p.cal_proior()

In [13]:
p.df[[p.id, p.sid, 'label']].groupby(p.id).first().reset_index().to_csv('result1.csv', index=False)

In [16]:
p.df[p.id].value_counts()

16171711    2634
453815      2088
927005      1715
394270      1575
873382       848
            ... 
1264709        1
977784         1
1026273        1
977011         1
1797           1
Name: Short Item No, Length: 22017, dtype: int64

In [71]:
p.df[p.df[p.id] == '16171711'][p.weight]

128       6.00
340       1.50
517       0.45
714       0.06
748       0.30
          ... 
203199    0.30
203409    0.24
203417    1.50
203458    0.06
203530    0.12
Name: Used Weight, Length: 2634, dtype: float64

In [36]:
p.df.iloc[[128, 340], 110:120]

,Pick Seq,Repl Seq,Move Code,Address Number,Ship To Number,Ln Ty,Priority (1/0),Carrier Number,Route Code,Stop Code
128,1061,NaN,NaN,65,65,S,5,1020012,T2,SE
340,1061,NaN,NaN,10,10,S,5,1020012,CUK,UK


In [53]:
volumn[[p.id, p.weight, p.cub]]

KeyError: "['Used Weight', 'Used Cubic Dim'] not in index"

In [48]:
volumn[volumn[p.id] == 16171711][[p.id, 'Gross Cubic Dim', 'Gross Weight']]

,Short Item No,Gross Cubic Dim,Gross Weight
134883,16171711.0,36.00,0.03


In [56]:
p.df.groupby(p.id).first().reset_index()

,Short Item No,Batch Number,Suggestion Sequence,T Y,Or Cd,Batch Number.1,Request Sequence,Wave Number,Task Number,Trip Number,...,Trip Depot,Line No..1,Parent License Plate Number,From License Plate Number,To License Plate Number,Job Number,Job Sequence,Delivery Batch Number,weight_score,label
0,100,26152790,1.000,2,SO,3179427,1.000,1247418,1250701,8.000,...,None,None,None,None,None,None,None,None,2.613674e-05,High
1,1000235,26449754,1.000,2,SO,3277044,4.000,1286507,1289857,4.000,...,None,None,None,None,None,None,None,None,1.956383e-09,low
2,1000253,26449755,1.000,2,SO,3277044,5.000,1286507,1289857,5.000,...,None,None,None,None,None,None,None,None,1.956383e-09,low
3,1000262,26449751,1.000,2,SO,3277044,1.000,1286507,1289857,1.000,...,None,None,None,None,None,None,None,None,1.956383e-09,low
4,1000264,26449753,1.000,2,SO,3277044,3.000,1286507,1289857,3.000,...,None,None,None,None,None,None,None,None,1.956383e-09,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22012,999905,26900791,1.000,2,SO,3436945,4.000,1345063,1348507,14.000,...,None,None,None,None,None,None,None,None,1.581181e-04,low
22013,999912,26689254,1.000,2,SO,3356519,1.000,1318489,1321885,5.000,...,None,None,None,None,None,None,None,None,2.881099e-04,low
22014,999957,26980521,1.000,2,SO,3465453,8.000,1355724,1359187,8.000,...,None,None,None,None,None,None,None,None,1.427018e-03,Medium
22015,999960,26980522,1.000,2,SO,3465453,9.000,1355724,1359187,9.000,...,None,None,None,None,None,None,None,None,1.427018e-03,Medium


In [59]:
tmp = pd.merge(p.df.groupby(p.id).first().reset_index(), volumn[[p.id, 'Gross Cubic Dim', 'Gross Weight']], on=p.id, how='left')

In [1]:
from src.docker import docker_main

In [3]:
from src.docker import GoodSequence
from src.process import Process
import pandas as pd

In [ ]:
result = docker_main()

In [7]:
p = Process()
p.clean_data()
weights = pd.read_csv(p.data_root / 'result1.csv', dtype=str)
volumns = pd.read_csv(p.data_root / 'Volume.csv', dtype=str)
weights = weights[weights['label'] != 'low'][p.id]
p.df = p.df[p.df[p.id].isin(weights)].reset_index(drop=True)
tmp = pd.merge(p.df.groupby(p.id).first().reset_index(), volumns[[p.id, 'Gross Cubic Dim', 'Gross Weight']], on=p.id, how='left') 
tmp = tmp.drop_duplicates(subset=[p.id]).dropna(subset=['Gross Cubic Dim', 'Gross Weight'])
tmp = tmp[[p.id, 'Gross Cubic Dim', 'Gross Weight']]
tmp['Gross Cubic Dim'] = tmp['Gross Cubic Dim'].str.replace(',', '').astype(float)
tmp['Gross Weight'] = tmp['Gross Weight'].str.replace(',', '').astype(float)

large = tmp[(tmp['Gross Weight'] > 10) | (tmp['Gross Cubic Dim'] > 6000)]
small = tmp[~tmp[p.id].isin(large[p.id])]
large.columns = ['name', 'height', 'weight']
small.columns = ['name', 'height', 'weight']

In [13]:
gs = GoodSequence(large)
gs.cal_mean_weight()
gs.forward(300)
result = gs.generate_dict()

In [ ]:
for idx in range(gs.docker_num):
    print(len(gs.dockers[idx].container))
    print(gs.dockers[idx].cap, gs.dockers[idx].total, gs.dockers[idx].total / gs.dockers[idx].row)

In [11]:
gs = GoodSequence(small)
gs.cal_mean_weight()
gs.forward(1)
result = gs.generate_dict()

In [6]:
result.to_csv('./result2.csv', index=False)

In [1]:
import pandas as pd
from src.process import Process
from src.optimize import Optimize

In [2]:
p = Process()
op = Optimize(p)

In [3]:
data = p.df.groupby(p.key)[p.id].agg(list)
it = op.get_association(data)

In [4]:
location = op.forward()

In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame(location.items())

In [28]:
location[it[0][0]]

(13, 34)

In [3]:
location = op.forward_with_association()

In [1]:
import pandas as pd

df1 = pd.read_csv('src/result1.csv')
df2 = pd.read_csv('src/result2.csv')
df3 = pd.read_csv('src/result3.csv')

In [10]:
pd.merge(pd.merge(df1, df2, on='Short Item No', how='left'), df3, on='Short Item No', how='left').sort_values('label').to_csv('result.csv', index=False)